In [1]:
# IMPORTING THE REQUIRED LIBRARIES

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

DATA PROCESSING

In [2]:
train=pd.read_csv('train.csv',encoding='latin-1')

In [3]:
train.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [4]:
train.tail()

,ItemID,Sentiment,SentimentText
99984,99996,0,@Cupcake seems like a repeating problem hop...
99985,99997,1,@cupcake__ arrrr we both replied to each other...
99986,99998,0,@CuPcAkE_2120 ya i thought so
99987,99999,1,@Cupcake_Dollie Yes. Yes. I'm glad you had mor...
99988,100000,1,@cupcake_kayla haha yes you do


In [5]:
# CLEANSING OF DATA

import re
# Removing URLS
def url(aao):
    text=aao[0]
    text=re.sub(r'http\S+','',text)
    return text

# Removing Usernames
def username(aao):
    text=aao[0]
    text=re.sub(r'@[A-Za-z0-9_]{1,15}','',text)
    return text

# Removing Special Characters and Digits
def special_digit(aao):
    text=aao[0]
    text=re.sub(r'[^A-Za-z ]+','',text)
    return text

# Removing extra Spaces
def space(aao):
    text=aao[0]
    text.strip()
    return text

# Converting every text to Lower Case
def lower(aao):
    text=aao[0]
    return text.lower()

In [6]:
train[['SentimentText']]=train[['SentimentText']].apply(url,axis=1)
train[['SentimentText']]=train[['SentimentText']].apply(username,axis=1)
train[['SentimentText']]=train[['SentimentText']].apply(special_digit,axis=1)
train[['SentimentText']]=train[['SentimentText']].apply(space,axis=1)
train[['SentimentText']]=train[['SentimentText']].apply(lower,axis=1)

In [7]:
train.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my apl friend
1,2,0,i missed the new moon trailer
2,3,1,omg its already o
3,4,0,omgaga im sooo im gunna cry ive be...
4,5,0,i think mi bf is cheating on me tt


In [8]:
train.tail()

,ItemID,Sentiment,SentimentText
99984,99996,0,seems like a repeating problem hope youre ...
99985,99997,1,arrrr we both replied to each other over diff...
99986,99998,0,ya i thought so
99987,99999,1,yes yes im glad you had more fun with me
99988,100000,1,haha yes you do


TEXT PROCESSING INVOLVING STEMMING AND TOKENISATION AND THEN CREATING BAG OF WORDS

In [9]:
import nltk

In [10]:
# STEMMING AND REMOVING THE STOPWORDS

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
ps=PorterStemmer()
words=[]
for i in range(0,99989):
    text=train['SentimentText'][i]
    text=text.split()
    text=[ps.stem(j) for j in text if not j in set(stopwords.words('english'))]
    text=' '.join(text)
    words.append(text)

In [11]:
# TOKENISATION AND CREATING BAG OF WORDS

from sklearn.feature_extraction.text import CountVectorizer 
cv=CountVectorizer(max_features=2500)
X=cv.fit_transform(words).toarray()
y=np.asarray(train['Sentiment'])

In [12]:
# SPLITTING DATASET INTO TRAIN AND TEST DATASET

from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
from sklearn.metrics import confusion_matrix,classification_report

LOGISTIC REGRESSION

In [15]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
pred=lr.predict(X_test)

In [17]:
print(confusion_matrix(y_test,pred))

[[ 8335  4825]
 [ 2800 14037]]


In [18]:
print(classification_report(y_test,pred))

             precision    recall  f1-score   support

          0       0.75      0.63      0.69     13160
          1       0.74      0.83      0.79     16837

avg / total       0.75      0.75      0.74     29997



NAIVE BAYES CLASSIFICATION

In [19]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(X_train,y_train)

GaussianNB(priors=None)

In [20]:
pred=nb.predict(X_test)

In [21]:
print(confusion_matrix(y_test,pred))

[[ 7656  5504]
 [ 4426 12411]]


In [22]:
print(classification_report(y_test,pred))

             precision    recall  f1-score   support

          0       0.63      0.58      0.61     13160
          1       0.69      0.74      0.71     16837

avg / total       0.67      0.67      0.67     29997



RANDOM FOREST CLASSIFICATION

In [24]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
pred=rfc.predict(X_test)

In [26]:
print(confusion_matrix(y_test,pred))

[[ 9049  4111]
 [ 4334 12503]]


In [27]:
print(classification_report(y_test,pred))

             precision    recall  f1-score   support

          0       0.68      0.69      0.68     13160
          1       0.75      0.74      0.75     16837

avg / total       0.72      0.72      0.72     29997

